<a href="https://colab.research.google.com/github/jrbalderrama/a2r2/blob/main/a2r2-01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RUDI Workshop: Introduction to Privacy-Preserving Data Publishing Techniques
=====================================================================

Notebook __ONE__
----------------

Tristan ALLARD & Javier ROJAS BALDERRAMA

_Univ Rennes, CNRS, INRIA_
  
This work is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/)

# Acknowledgments

We warmly thank François Bodin and Luc Lesoil for their support on the data and the use case.


# Step 0 (STARTER) : 

This hands-on tutorial is going to introduce you to the issue of *privacy-preserving personal data publishing*. You are going to follow the implementation of a concrete use-case built from open data from the Rennes Metropole area. The main question of the use-case is to know wether a change in the students schedules at the Beaulieu campus impacts the load of the buses that go through the campus. We will answer to this question based on two datasets : the validations inside the buses that stop close to the campus (with timestamps), and the number of students that terminate a class (with timestamps). Our approach consists in training a predictor that outputs the expected number of validations along the day given the number of students terminating a class along the day. However, using raw buses validations for answering to this question may lead to privacy issues because validations can be highly identifying. After having performed some reidentification attacks, you will use a perturbed version of the buses validations dataset and observe the resulting impact on our ability to answer to the main question of the use-case. 

We designed this tutorial to be a step-by-step guided tour. You can follow sequentially the "Step i" tag inside the titles of the sections. Up to you to follow the sequence proposed or to deviate from it, but be careful when leaving the track, it's wild out there ;) 

We divided the full journey into three topics :
    1. The naive version
    2. Privacy issues
    3. The protected version
    
For convenience, one notebook is dedicated to each topic and **you are currently in Notebook 1.**

We also ask questions. Please take the time to think about them. Trying to answer them can also help you gain a deeper understanding. And we would love reading your answers! 

Ready? 

Really?? 

**You can now go to Step 1!**



# Step 2 (PREAMBLE) : Settings and Data

Not too disappointed ? So lets now have a look at the data based on which we trained the model. 

1.   The datasets are downloaded 
2.   The libraries required are imported and global variables are setup
3.   The raw data are aggregated...
4.   ... And the results are displayed. 
5.   The datasets are prepared for the training process.

Observe the buses validations dataset (Section "Display raw data")... Can you imagine any issue ? 


 ## Data download


In [1]:
# Download the data : students counts and buses validations.

!wget -nv -nc https://zenodo.org/record/5509313/files/classes.parquet
!wget -nv -nc https://zenodo.org/record/5509268/files/buses.parquet

2021-09-22 10:33:57 URL:https://zenodo.org/record/5509313/files/classes.parquet [17736/17736] -> "classes.parquet" [1]
2021-09-22 10:33:58 URL:https://zenodo.org/record/5509268/files/buses.parquet [3594503/3594503] -> "buses.parquet" [1]


 ## Required imports and setups

In [2]:
# Import libraries
import itertools
import math
import os
from datetime import datetime
from errno import ENOENT
from pathlib import Path
from typing import Optional, Sequence, Tuple

import folium
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import pyarrow.parquet as pq
import torch
from folium.plugins import HeatMapWithTime
from numpy import linalg, ndarray
from pandas import NA, DataFrame, DatetimeIndex, Series, Timedelta, Timestamp
from plotly import subplots
from plotly.graph_objs import Bar, Candlestick, Figure, Scatter
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from torch import Tensor
from torch.nn import LSTM, Linear, Module, MSELoss
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

ModuleNotFoundError: No module named 'pyarrow'

In [ ]:
# Set rendered for image output
from IPython import get_ipython
if "google.colab" in str(get_ipython()):
    pio.renderers.default = "colab"

In [ ]:
# Set global options and define notebook constants

# project base directory
BASE_DIRECTORY = Path(".")


## Raw data preparation

### Read raw data

In [ ]:
# Read data from file system and plot data

# load dataset from file system
def load_data(
    path: Path,
) -> DataFrame:
    if not path.exists():
        raise FileNotFoundError(ENOENT, os.strerror(ENOENT), path)

    table = pq.read_table(path)
    return table.to_pandas()


# buses dataset
buses_filename = "buses.parquet"
buses_path = BASE_DIRECTORY.joinpath(buses_filename)
buses_dataset = load_data(buses_path)


# classes dataset
classes_filename = "classes.parquet"
classes_path = BASE_DIRECTORY.joinpath(classes_filename)
classes_dataset = load_data(classes_path)

#from google.colab import data_table
#data_table.DataTable(buses_dataset[:20000], include_index=False, num_rows_per_page=20)

### Display raw data

In [ ]:

#from google.colab import data_table
#data_table.DataTable(buses_dataset[:20000], include_index=False, num_rows_per_page=20)
####################
# BEGIN : Observe

In [ ]:
# END : Observe
####################

In [ ]:
####################
# BEGIN : Answer

Food for thoughts: 
   1. Is there any information directly identifying in the raw data ? 
   2. Could you describe possible auxiliary information that could lead to re-identifications ?

In [ ]:
# END : Answer
####################

**You can now go to Notebook 2.**

### Agregate raw data

In [ ]:
# Post-processing input data

# post processing (aggregated) transportation data
def post_processing_by_aggregation(
    dataframe: DataFrame,
    *,
    stops: Optional[Sequence[str]],
    ignore_weekend: bool = False,
) -> DataFrame:

    dataframe_ = dataframe.copy()
    # filter data from 'bus_stops' only
    if stops:
        dataframe_ = dataframe_[dataframe_["stop_name"].isin(beaulieu)]

    # remove weekend information
    if ignore_weekend:
        dataframe_ = dataframe_.set_index("departure_time")
        dataframe_ = dataframe_[dataframe_.index.dayofweek < 5]

    # aggregate dataset by stop name and departure time
    dataframe_ = (
        dataframe_.groupby(
            [
                "stop_name",
                "departure_time",
            ]
        )
        .agg({"count": "sum"})
        .reset_index()
    )

    return dataframe_.groupby("departure_time").sum()


def plot_dataset(
    dataframe: DataFrame,
    column: str,
) -> None:
    figure = Figure()
    scatter = Scatter(
        x=dataframe.index,
        y=dataframe[column],
        mode="lines",
        name="values",
    )

    figure.add_trace(scatter)
    figure.update_layout(
        showlegend=False,
        title_text=column,
        template="simple_white",
    )

    figure.update_xaxes(showgrid=True)
    figure.show()

In [ ]:
# target bus stops
beaulieu = [
    "Les Préales",
    "Tournebride",
    "Beaulieu Chimie",
    "Beaulieu INSA",
    "Beaulieu Restau U",
]



buses_dataset = post_processing_by_aggregation(
    buses_dataset,
    stops=beaulieu,
)

### Display agregated data

#### Number of validations

In [ ]:
display(buses_dataset)
plot_dataset(buses_dataset, "count")

#### Number of students

In [ ]:
display(classes_dataset)
plot_dataset(classes_dataset, "nombre_etudiant")

In [ ]:
# Merge datasets
def merge_datasets(
    classes: DataFrame,
    buses: DataFrame,
) -> DataFrame:

    # ignore dataset entries that are not available in classes timeline
    buses_ = buses[
        buses.index
        <= classes.index.max()
        + Timedelta(
            1,
            unit="day",
        )
    ]

    # merge datasets
    dataset = pd.merge(
        classes,
        buses_,
        how="outer",
        left_index=True,
        right_index=True,
    )

    # fill empty values
    dataset = dataset.fillna(0)

    return dataset

#### Display the merged dataset (students and buses)

In [ ]:
dataset = merge_datasets(classes_dataset, buses_dataset)
display(dataset)

## Data preparation for the neural network

### Make dates and times understandable for our neural network

In [ ]:
# Add features (motifs) to the dataset
la_rentree = Timestamp("2021-09-06")
la_toussaint = Timestamp("2021-11-01")
one_week_timedelta = Timedelta(7, unit="day")

# bucketize attribute
def onehot_encode(
    dataframe: DataFrame,
    column: str,
) -> DataFrame:
    dummies = pd.get_dummies(
        dataframe[column],
        prefix=column,
    )

    return pd.concat(
        [dataframe, dummies],
        axis=1,
    ).drop(columns=[column])


# encode (time) column as periodic wave
def periodic_encode(
    dataframe: DataFrame,
    column: str,
    period: int,
    start_num: int = 0,
) -> DataFrame:
    kwargs = {
        f"sin_{column}": lambda x: np.sin(
            2 * np.pi * (dataframe[column] - start_num) / period
        ),
        f"cos_{column}": lambda x: np.cos(
            2 * np.pi * (dataframe[column] - start_num) / period
        ),
    }

    return dataframe.assign(**kwargs).drop(columns=[column])


# add uniform timeindex column
def set_time_index(
    dataframe: DataFrame,
    frequence: int = 15,
) -> DataFrame:
    dataframe_ = dataframe.copy()

    # add a time index using the frequency
    dataframe_["time_idx"] = dataframe_.index - dataframe_.index.min()
    dataframe_["time_idx"] = (
        dataframe_["time_idx"].astype("timedelta64[m]") // frequence
    )
    dataframe_["time_idx"] = dataframe_["time_idx"].astype("int_")
    return dataframe_


# mark dataset ranges as holidays
def label_holidays(
    dataframe: DataFrame,
    start: Timestamp,
    end: Timestamp,
    column="holiday",
) -> DataFrame:
    dataframe_ = dataframe.copy()
    dataframe_[column] = 0
    dataframe_.loc[
        (dataframe_.index >= start) & (dataframe_.index < end),
        column,
    ] = 1
    return dataframe_


# generate lags (to track interaction throughout time)
def generate_lags(
    dataframe: DataFrame,
    lags: int,
    column: str,
) -> DataFrame:
    dataframe_ = dataframe.copy()
    for n in range(1, lags + 1):
        dataframe_[f"{column}_lag_{n}"] = dataframe_[column].shift(n)

    return dataframe_.fillna(0)


# add features to the dataset
def add_features(
    dataframe: DataFrame,
    bucketize_date: bool = True,
    periodic_time: bool = True,
    holidays: bool = False,
    timeindex: bool = False,
    lags: bool = False,
    n_lags: int = 50,
) -> DataFrame:
    dataframe_ = dataframe.copy()
    if timeindex:
        dataframe_ = set_time_index(dataframe_)

    if bucketize_date:
        dataframe_ = dataframe_.assign(dayofweek=dataframe_.index.dayofweek)
        # .assign(day=dataframe.index.day)
        # .assign(month=dataset.index.month)
        dataframe_ = onehot_encode(dataframe_, "dayofweek")
        # dataset = onehot_encode(dataset, "month")

    if periodic_time:
        dataframe_ = dataframe_.assign(hour=dataframe_.index.hour)
        dataframe_ = dataframe_.assign(minute=dataframe_.index.minute)
        dataframe_ = periodic_encode(dataframe_, "hour", 24, 0)
        dataframe_ = periodic_encode(dataframe_, "minute", 60, 0)

    if holidays:
        dataframe_ = label_holidays(
            dataframe_,
            la_toussaint,
            la_toussaint + one_week_timedelta,
        )

    if lags:
        dataframe_ = generate_lags(dataframe_, n_lags, "count")
        dataframe_ = generate_lags(dataframe_, n_lags, "nombre_etudiant")

    # dataframe.drop(["nombre_etudiant"], axis=1, inplace=True)
    return dataframe_

 ### Displaying the data formatted for the machine learning process

In [ ]:
dataset = add_features(dataset, holidays=True)
display(dataset)

### Split the dataset into the train, the test, and the validation subsets

In [ ]:
def plot_timeline2(
    dataframe: DataFrame,
    columns: Sequence[str],
    delimiters: Sequence[Timestamp],
    holidays: Tuple[Timestamp, Timestamp],
) -> None:
    dmin = dataframe["nombre_etudiant"].values.min()
    dmax = dataframe["nombre_etudiant"].values.max()
    figure = subplots.make_subplots(specs=[[{"secondary_y": True}]])
    for counter, column in enumerate(columns):
        secondary_y = False if counter % 2 == 0 else True
        scatter = Scatter(
            x=dataframe.index,
            y=dataframe[column],
            mode="lines",
            name=column,
        )
      
        figure.add_trace(
            scatter,
            secondary_y=secondary_y,
        )
      
    for delimiter in delimiters:
        figure.add_shape(
            type="line",
            x0=delimiter,
            x1=delimiter,
            y0=dmax,
            y1=0,
            line=dict(
                # color="Gray",
                width=1,                 
                dash="dashdot",
            ),    
        )

    figure.add_shape(
        type="rect",
        xref="paper",
        yref="paper",
        layer="below",
        fillcolor="LightSeaGreen",
        x0=holidays[0],
        x1=holidays[1],
        y0=dmax,
        y1=0,
    )

    figure.add_annotation(
        x=holidays[0],
        y=dmax,
        align="right",
        text="holidays",
        showarrow=False,
        yshift=-25,
        textangle=90,
        xshift=10,
    )
                                                                                    
    figure.add_annotation(
        x=delimiters[0],
        y=dmax,
        text="validation",
        showarrow=True,
         yshift=-15,
    )

    figure.add_annotation(
        x=delimiters[1],
        y=dmax,
        text="test",
        showarrow=True,
    )

    figure.update_shapes(dict(xref="x", yref="y"))
    figure.update_yaxes(
        rangemode="tozero",
        # type="log",
        )

    figure.update_xaxes(range=[dataframe.index.min(), dataframe.index.max()])
    figure.update_yaxes(title_text=columns[0], secondary_y=False)
    figure.update_yaxes(title_text=columns[1], secondary_y=True)            
    figure.update_layout(
        title_text="Count of Buses & Classes",
        template="simple_white",  
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )     
    )

    figure.show()

In [ ]:
# Split the data into test, validation, and train sets
def features_split(
    dataframe: DataFrame,
    target: str,
) -> Tuple[DataFrame, DataFrame]:
    y = dataframe[[target]]
    X = dataframe.drop(columns=[target])
    return X, y


def get_timestamp_bound(
    dataframe: DataFrame,
    weeks: int,
) -> Timestamp:
    timedelta = Timedelta(7 * weeks - 1, unit="day")
    timestamp = dataframe.index.min() + timedelta
    return timestamp.normalize()



def plot_timeline(
    dataframe: DataFrame,
    columns: Sequence[str],
    delimiters: Sequence[Timestamp],
    holidays: Tuple[Timestamp, Timestamp],
) -> None:
    dmin = dataframe["nombre_etudiant"].values.min()
    dmax = dataframe["nombre_etudiant"].values.max()
    figure = Figure()
    for column in columns:
        scatter = Scatter(
            x=dataframe.index,
            y=dataframe[column],
            mode="lines",
            name=column,
        )
        figure.add_trace(scatter)

    for delimiter in delimiters:
        figure.add_trace(
            Scatter(
                x=[delimiter, delimiter],
                y=[dmin, dmax],
                mode="lines",
                name="end",
                line=dict(color="black", width=1, dash="dashdot"),
            )
        )

    figure.add_shape(
        line_color="yellow",
        fillcolor="LightSeaGreen",
        opacity=0.3,        
        x0=holidays[0],
        x1=holidays[1],
        layer="below",
        y0=dmax,
        y1=0,
        xref="x",
        yref="y",
    )

    figure.update_yaxes(type="log", range=[0, 4])
    figure.update_layout(
        showlegend=False,
        title_text="Count of Buses & Classes",
        template="simple_white",
    )

    figure.show()

              

### Displaying the train, test, validation subsets

In [ ]:
end_train = get_timestamp_bound(dataset, weeks=9)
end_val = get_timestamp_bound(dataset, weeks=10)

# display(dataset)
plot_timeline2(
    dataset,
    ["nombre_etudiant", "count"],
    [end_train, end_val],
    (la_toussaint, la_toussaint + one_week_timedelta),
)

train_dataset = dataset[dataset.index < end_train]
val_dataset = dataset[(dataset.index >= end_train) & (dataset.index < end_val)]
test_dataset = dataset[dataset.index >= end_val]

X_train, y_train = features_split(train_dataset, target="count")
X_val, y_val = features_split(val_dataset, target="count")
X_test, y_test = features_split(test_dataset, target="count")

# TOOL: a neural network

## Defining our neural network


In [ ]:
# Define and run a RNN model
class LSTMModel(Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout):
        super().__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # LSTM layers
        self.lstm = LSTM(
            input_dim,
            hidden_dim,
            layer_dim,
            batch_first=True,
            dropout=dropout,
        )

        # Fully connected layer
        self.fc = Linear(hidden_dim, output_dim)

    def forward(self, x):
        # initializing hidden state for first input with zeros
        h0 = torch.zeros(
            self.layer_dim,
            x.size(0),
            self.hidden_dim,
        ).requires_grad_()

        # initializing cell state for first input with zeros
        c0 = torch.zeros(
            self.layer_dim,
            x.size(0),
            self.hidden_dim,
        ).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        # Forward propagation by passing in the input, hidden state, and cell state into the model
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        # (squeezing is equivalent to: `out = out[:, -1, :]`)
        out = torch.squeeze(out)

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out


class RunnerHelper:
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []

    def train_step(self, X, y):

        # set model to train mode
        self.model.train()

        # make predictions
        ŷ = self.model(X)

        # compute loss
        loss = self.loss_fn(ŷ, y)

        # compute gradients
        loss.backward()

        # update parameters
        self.optimizer.step()

        # reset to zero gradients
        self.optimizer.zero_grad()

        # returns loss
        return loss.item()

    def val_step(self, X, y):

        # set model to eval mode
        self.model.eval()

        # make prediction
        ŷ = self.model(X)

        # compute loss
        loss = self.loss_fn(ŷ, y)

        # return loss
        return loss.item()

    def train(self, train_loader, val_loader, n_epochs=50):
        model_path = f'{self.model}_{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
        for epoch in range(1, n_epochs + 1):
            batch_train_losses = []
            for x_train, y_train in train_loader:
                # x_train = x_train.view([batch_size, -1, n_features]).to(DEVICE)
                x_train = torch.unsqueeze(x_train, 1)
                train_loss = self.train_step(x_train, y_train)
                batch_train_losses.append(train_loss)

            training_loss = np.mean(batch_train_losses)
            self.train_losses.append(training_loss)
            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    # x_val = x_val.view([batch_size, -1, n_features]).to(DEVICE)
                    x_val = torch.unsqueeze(x_val, 1)
                    val_loss = self.val_step(x_val, y_val)
                    batch_val_losses.append(val_loss)

                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)

            if (epoch <= 10) | (epoch % 20 == 0):
                print(
                    f"[{epoch:3d}/{n_epochs}] Training loss: {training_loss:.4f}"
                    f"\t Validation loss: {validation_loss:.4f}"
                )

        # torch.save(self.model.state_dict(), model_path)

    def evaluate(self, test_loader):
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:
                # x_test = x_test.view([batch_size, -1, n_features]).to(DEVICE)
                x_test = torch.unsqueeze(x_test, 1)
                self.model.eval()
                ŷ = self.model(x_test)
                predictions.append(ŷ.detach().numpy())
                values.append(y_test.detach().numpy())

        return predictions, values

    def plot_losses(self):
        figure = Figure()
        tics = [*range(len(self.train_losses) + 1)]
        value = Scatter(
            x=tics,
            y=self.train_losses,
            mode="lines",
            name="Training",
            marker=dict(),
        )

        figure.add_trace(value)
        value = Scatter(
            x=tics,
            y=self.val_losses,
            mode="lines",
            name="Validation",
            marker=dict(),
        )

        figure.add_trace(value)
        figure.update_layout(title_text="Losses")
        figure.show()


# rescale results and align it to original time index
def inverse_transform(
    values: Sequence[ndarray],
    predictions: Sequence[ndarray],
    index: DatetimeIndex,
    scaler: MinMaxScaler,
) -> DataFrame:
    vals = np.concatenate(values, axis=0).ravel()
    preds = np.concatenate(predictions, axis=0).ravel()
    dataframe = DataFrame(
        data={
            "value": vals,
            "prediction": preds,
        },
        index=index[: len(vals)],
    )

    dataframe = dataframe.sort_index()
    dataframe = DataFrame(
        scaler.inverse_transform(dataframe),
        columns=dataframe.columns,
        index=dataframe.index,
    )

    return dataframe.astype("int_")


def print_metrics(
    dataframe: DataFrame,
    value: str,
    prediction: str = "prediction",
) -> None:
    result_metrics = {
        "mae": metrics.mean_absolute_error(
            dataframe[value],
            dataframe[prediction],
        ),
        "rmse": metrics.mean_squared_error(
            dataframe[value],
            dataframe[prediction],
        )
        ** 0.5,
        "r2": metrics.r2_score(
            dataframe[value],
            dataframe[prediction],
        ),
    }

    print("\tMean Absolute Error:       ", result_metrics["mae"])
    print("\tRoot Mean Squared Error:   ", result_metrics["rmse"])
    print("\tR^2 Score:                 ", result_metrics["r2"])
    # return result_metrics


# show residuals as kind of OHLC Charts
def plot_residuals(
    dataframe: DataFrame,
) -> None:
    hovertext = []
    for i in range(dataframe.shape[0]):
        hovertext.append(
            f"{dataframe.index[i]}<br>"
            f"Real: {dataframe['value'][i]}<br>"
            f"Prediction: {dataframe['prediction'][i]}"
        )

    figure = Figure(
        data=[
            Scatter(
                x=dataframe.index,
                y=dataframe["value"],
                mode="lines",
                name="reference",
                line=dict(color="lightgrey", width=0.6, dash="dot"),
                # opacity=0.6,
                showlegend=False,
            ),
            Scatter(
                x=dataframe.index,
                y=dataframe["prediction"],
                mode="lines",
                name="prediction",
                line=dict(color="lightblue", width=0.6, dash="dot"),
                showlegend=False,
                # opacity=0.6,
            ),
            Candlestick(
                x=dataframe.index,
                open=dataframe["value"],
                high=dataframe["prediction"],
                low=dataframe["prediction"],
                close=dataframe["value"],
                text=hovertext,
                hoverinfo="text",
                name="residuals",
                # line=dict(width=2),
                increasing_line_color="lightseagreen",
                decreasing_line_color="lightsalmon",
            ),
        ]
    )

    figure.update_layout(
        title="Prediction residuals",
        template="simple_white",
        xaxis_rangeslider_visible=True,
    )

    figure.show()


# formating data for NN
def to_dataloaders(
    dataframe_train: Tuple[DataFrame, DataFrame],
    dataframe_val: Tuple[DataFrame, DataFrame],
    dataframe_test: Tuple[DataFrame, DataFrame],
    scaler: MinMaxScaler,
    batch_size,
    shuffle=False,
    drop_last=True,
) -> Tuple[DataLoader, DataLoader, DataLoader]:

    # scale data
    X_train_arr = scaler.fit_transform(dataframe_train[0])
    X_val_arr = scaler.transform(dataframe_val[0])
    X_test_arr = scaler.transform(dataframe_test[0])

    y_train_arr = scaler.fit_transform(dataframe_train[1])
    y_val_arr = scaler.transform(dataframe_val[1])
    y_test_arr = scaler.transform(dataframe_test[1])

    # transform scaled data to tensors
    train_features = Tensor(X_train_arr)
    train_targets = Tensor(y_train_arr)
    val_features = Tensor(X_val_arr)
    val_targets = Tensor(y_val_arr)
    test_features = Tensor(X_test_arr)
    test_targets = Tensor(y_test_arr)

    # setup tensor datasets
    train = TensorDataset(train_features, train_targets)
    val = TensorDataset(val_features, val_targets)
    test = TensorDataset(test_features, test_targets)

    # setup (tensor) datasets loaders
    train_loader = DataLoader(
        train,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
    )

    val_loader = DataLoader(
        val,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
    )

    test_loader = DataLoader(
        test,
        batch_size=1,
        shuffle=shuffle,
        drop_last=drop_last,
    )

    return train_loader, val_loader, test_loader

## Parameterizing our neural network

In [ ]:
# NN parameters
HIDDEN_DIM = 64
LAYER_DIM = 3
BATCH_SIZE = 64
EPOCHS = 100

## Training our neural network

In [ ]:
input_dim = len(X_train.columns)  # X_train.shape[0]
model = LSTMModel(
    input_dim=input_dim,
    hidden_dim=HIDDEN_DIM,
    layer_dim=LAYER_DIM,
    output_dim=1,
    dropout=0.2,
)

scaler = MinMaxScaler()  # RobustScaler()  # StandardScaler()  # MinMaxScaler()
loss_fn = MSELoss()  # L1Loss()
optimizer = Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
runner = RunnerHelper(model=model, loss_fn=loss_fn, optimizer=optimizer)
train_loader, val_loader, test_loader = to_dataloaders(
    (X_train, y_train),
    (X_val, y_val),
    (X_test, y_test),    
    scaler,
    BATCH_SIZE,
)

runner.train(train_loader, val_loader, n_epochs=EPOCHS)
runner.plot_losses()
predictions, values = runner.evaluate(test_loader)
lstm_result = inverse_transform(values, predictions, X_test.index, scaler)

## Visualising the quality of our neural networks

In [ ]:
print(f"NN model: LSTM")
print_metrics(lstm_result, "value")
display(lstm_result)
plot_residuals(lstm_result)

## Comparing our neural network against a baseline method

 ### Train a linear regression model

In [ ]:
# Build a baseline model to compare against the RNN model
def baseline_evaluate(
    X_train: DataFrame,
    y_train: DataFrame,
    X_test: DataFrame,
    y_test: DataFrame,
) -> DataFrame:
    model = LinearRegression()
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    dataframe = DataFrame(y_test)
    dataframe = dataframe.assign(prediction=prediction)
    dataframe = dataframe.sort_index()
    return dataframe


def plot_models_prediction_interval(
    dataframe: DataFrame,
    rnn_dataframe: DataFrame,
    baseline_dataframe: DataFrame,
) -> None:
    figure = Figure()
    value = Scatter(
        x=dataframe.index,
        y=dataframe["count"],
        mode="lines",
        name="Reference",
        line=dict(color="rgba(0,0,0, 0.3)", width=1, dash="dot"),
    )

    figure.add_trace(value)
    baseline = Scatter(
        x=baseline_dataframe.index,
        y=baseline_dataframe.prediction,
        mode="lines",
        name="Linear Regression",
        opacity=0.8,
    )

    figure.add_trace(baseline)
    prediction = Scatter(
        x=rnn_dataframe.index,
        y=rnn_dataframe.prediction,
        mode="lines",
        name="LSTM NN",
        # marker=dict(),
        opacity=0.8,
        visible="legendonly",
    )

    figure.add_trace(prediction)
    figure.update_layout(
        showlegend=True,
        title_text="Predictions",
        template="simple_white",
        xaxis=dict(
            range=[
                rnn_dataframe.index.min(),
                rnn_dataframe.index.max(),
            ],
        ),
    )

    figure.update_xaxes(rangeslider_visible=True)
    figure.show()

### Visualize the predictions of the two models

In [ ]:
print("Baseline model: linear regression")
baseline_result = baseline_evaluate(X_train, y_train, X_test, y_test)
print_metrics(baseline_result, "count")
plot_models_prediction_interval(dataset, lstm_result, baseline_result)

# Step 1 (RESULT) : impact of changing the students schedules on the buses validations

Lets start with the end. We are going to answer to the question raised by our use case : 
> **Could a change in the time at which students 
finish have a *significant* impact on the number of validations in buses ?**

In order to answer to this question, we have trained above a machine learning model that we are going to use as a predictor *(please wait a little bit for information on the training process)*. Given a time (and possibly a group of students), the model outputs an estimation of the number of buses validations on the campus. 

You can play with the timeshift below and observe the impact on the validations. Search the following comments : 
```
####################
# BEGIN : ...
...
# END : ...
####################
```






In [ ]:
# Test predictions with classes time shift
def shift_time(
    dataframe: DataFrame,
    minutes: int,
) -> Series:
    dataframe_ = dataframe.copy(deep=True)
    timedelta = Timedelta(minutes, unit="T")
    dataframe_.reset_index(inplace=True)
    dataframe_.iloc[:, [0]] += timedelta
    dataframe_.set_index(dataframe_.columns[0], inplace=True)
    return dataframe_


def plot_prediction_interval_with_staggings(
    dataframe: DataFrame,
    staggered: DataFrame,
) -> None:
    figure = subplots.make_subplots(
        rows=4,
        cols=1,
        shared_xaxes=True,
        specs=[
            [{"rowspan": 3}],
            [None],
            [{}],
            [{}],
        ],
        vertical_spacing=0.1,
    )

    prediction_plot = Scatter(
        x=dataframe.index,
        y=dataframe.prediction,
        mode="lines",
        name="prediction",
        # opacity=0.1,
        fill=None,
        showlegend=False,
        # line_color="gray",
        line=dict(color="gray", width=0.1),
        # hoverinfo="x+y",
        # stackgroup='one'
    )

    figure.add_trace(prediction_plot, row=1, col=1)
    staggered_plot = Scatter(
        x=staggered.index,
        y=staggered.prediction,
        mode="lines",
        name="staggered",
        # opacity=0.8,
        fill="tonexty",
        fillcolor="red",
        line=dict(color="gray", width=0.1),
        # hoverinfo="x+y",
        # stackgroup='one'
    )

    figure.add_trace(staggered_plot, row=1, col=1)
    residuals = (
        pd.merge(
            lstm_result,
            staggered_lstm_result,
            how="outer",
            left_index=True,
            right_index=True,
        )
        .rename(
            {
                "prediction_x": "prediction",
                "prediction_y": "staggered",
            },
            axis=1,
        )
        .drop(["value_x", "value_y"], axis=1)
        .dropna()
        .astype(int)
    )

    residuals["difference"] = residuals["prediction"] - residuals["staggered"]
    colors = [
        "lightseagreen" if c > 0 else "lightsalmon" for c in residuals["difference"]
    ]
    bar_plot = Bar(
        x=residuals.index,
        y=residuals.difference,
        name="difference",
        showlegend=False,
        marker_color=colors,
    )

    figure.add_trace(bar_plot, row=4, col=1)
    figure.update_xaxes(showticklabels=True, row=1, col=1)
    figure.update_yaxes(title_text="difference", row=4, col=1, zeroline=True, zerolinecolor="gray")
    figure.update_xaxes(
        showticklabels=False,
        visible=False,
        row=4,
        col=1,
    )

    figure.update_layout(
        showlegend=True,
        title_text="Predictions and Staggings",
        template="simple_white",
    )

    figure.show()


def evaluate_shift_time(
    buses: DataFrame,
    classes: DataFrame,
    runner: RunnerHelper,
    scaler: MinMaxScaler,
    test_bound: Timestamp,
    *,
    minutes: int,
) -> DataFrame:
    staggered_classes = shift_time(classes, minutes=minutes)
    dataframe = merge_datasets(staggered_classes, buses)
    dataframe = add_features(dataframe, holidays=True)
    test_dataset = dataframe[dataframe.index >= test_bound]
    X_test, y_test = features_split(
        test_dataset,
        target="count",
    )

    _, _, test_loader = to_dataloaders(
        (X_train, y_train),
        (X_val, y_val),
        (X_test, y_test),
        scaler,
        BATCH_SIZE,
    )

    predictions, values = runner.evaluate(test_loader)
    staggered_lstm_result = inverse_transform(
        values,
        predictions,
        X_test.index,
        scaler,
    )

    return staggered_lstm_result

In [ ]:
####################
# BEGIN : play

SHIFT_IN_MINUTES = 95

# END : play  
####################

In [ ]:
# reload original buses dataset for iterative modifications
buses_dataset = load_data(buses_path)
buses_dataset = post_processing_by_aggregation(
    buses_dataset,
    stops=beaulieu,
)

staggered_lstm_result = evaluate_shift_time(
    buses_dataset,
    classes_dataset,
    runner,
    scaler,
    end_val,
    minutes=SHIFT_IN_MINUTES,
)

plot_prediction_interval_with_staggings(
    lstm_result,
    staggered_lstm_result,
)

####################
# BEGIN : Observe

In [ ]:
# END : Observe
####################

In [ ]:
####################
# BEGIN : Answer

Food for thoughts: 
    1. How can you observe the impact of changing the schedules ? 
    2. What is the expected impact of shifting the schedules by 15mins ? 
    3. Is the expected impact of a 60 mins shift bigger ?
    4. Is there a _small_ shift (e.g., less than 60 mins) that would result in a large impact ? 

In [ ]:
# END : Answer
####################

**You can now go to Step 2 (same notebook).**

# Step 3 (ATTACK): the case for privacy

**(Switch to Notebook 2 please)**

Yes, raw data is not immune to reidentification ! 

You are now going to perform a reidentification attack on a small set of targets. To this end, we will give you some auxiliary information (also called background knowledge) and programming tools for helping you query the dataset. 

But first lets visualize 

## Displaying raw buses validations

In [ ]:
# Privacy metrics

# show detailed dataset
def display_heatmap_with_time(
    dataframe: DataFrame,
    group_column: str = "departure_time",
    # Rennes GPS coordinates
    location: Tuple[float, float] = (48.1147, -1.6794),
) -> None:
    _dataframe = dataframe.copy(deep=True)
    timestamps = []
    coordinates = []
    for timestamp, coordinate in _dataframe.groupby(group_column):
        timestamps.append(str(timestamp))
        coordinates.append(
            coordinate[
                [
                    "stop_lat",
                    "stop_lon",
                ]
            ].values.tolist()
        )

    base_map = folium.Map(
        location=location,
        zoom_start=11,
        tiles="https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png",
        # tiles="https://{s}.basemaps.cartocdn.com/dark_nolabels/{z}/{x}/{y}{r}.png",
        attr="CartoDB",
    )

    heat_map = HeatMapWithTime(
        data=coordinates,
        index=timestamps,
        auto_play=True,
        min_speed=1,
        radius=4,
        max_opacity=0.5,
    )

    heat_map.add_to(base_map)
    display(base_map)


def get_metrics_dataset(
    path: Path,
) -> DataFrame:
    dataframe = load_data(path)
    
    # drop geospatial attributes from dataset
    return dataframe[
        [
            "departure_time",
            "id",
            "stop_name",
            "route_short_name",
            "stop_id",
            "direction_id",
        ]
    ]

In [ ]:
buses_dataset = load_data(buses_path)
display_heatmap_with_time(buses_dataset)

## Attacking raw buses validations

TODO

## Explaining the success of the attacks

### Shannon's entropy

In [ ]:
# Shannon's entropy
def entropy(
    series: Series,
    base: int = 2,
    normalize: bool = False,
) -> float:
    def expectation(probability: Series) -> float:
        return (probability * np.log(probability) / np.log(base)).sum()

    def efficiency(entropy: float, length: int) -> float:
        return entropy * np.log(base) / np.log(length)

    probability = series.value_counts(normalize=True, sort=False)
    h = -expectation(probability)
    return efficiency(h, series.size) if normalize else h


def get_entropies(
    dataframe: DataFrame,
    base: int = 2,
    normalize: bool = False,
) -> Series:
    dataframe_ = dataframe.copy()
    entropies = dataframe_.apply(
        entropy,
        base=base,
        normalize=normalize,
    )

    return (
        entropies.to_frame()
        .reset_index()
        .rename(
            {
                "index": "attribute",
                0: "entropy",
            },
            axis=1,
        )
    )


def plot_entropies(
    dataframe: DataFrame,
) -> None:
    figure = px.bar(
        dataframe,
        x="entropy",
        y="attribute",
        orientation="h",
        color="attribute",
        template="plotly_white",
    )

    figure.update_traces(
        texttemplate="%{x:.2f}",
        textposition="auto",
    )

    figure.update_layout(showlegend=False)

    figure.show()

In [ ]:
dataset = get_metrics_dataset(buses_path)
display(dataset)
entropies = get_entropies(dataset, normalize=True)
plot_entropies(entropies)

### Anonymity Set

In [ ]:
# Anonymity set
def get_anonymity_set(
    dataframe: DataFrame,
    subset: Optional[Sequence[str]] = None,
    reindex: bool = False,
) -> Series:
    def reset_index(serie: Series) -> Series:
        domain = range(1, serie.index.max() + 1)
        return serie.reindex(domain, fill_value=0)

    dataframe_ = dataframe.copy()
    multiplicity = dataframe_.value_counts(subset=subset)
    aset = multiplicity.value_counts().sort_index()
    aset = reset_index(aset) if reindex else aset
    return (
        aset.to_frame()
        .reset_index()
        .rename(
            {
                "index": "cardinality",
                0: "occurrences",
            },
            axis=1,
        )
    )


def plot_anonymity_set(
    dataframe: DataFrame,
) -> None:
    figure = px.bar(
        dataframe,
        x="cardinality",
        y="occurrences",
        color="occurrences",
        color_continuous_scale="Bluered",
        # template="plotly_white",
        title="Anonymity Set",
    )

    figure.update_coloraxes(showscale=False)
    figure.show()

In [ ]:
dataset = get_metrics_dataset(buses_path)
anonymity_set = get_anonymity_set(
    dataset,
    subset=[
        "id",
        "stop_name",
        "route_short_name",
    ],
)

plot_anonymity_set(anonymity_set)

# SOLUTION: Sound protection with differential privacy

In [ ]:
# Perturb timeline series with differential privacy
def fpa(Q: ndarray, δ: float, ε: float, k: int) -> ndarray:

    # define a laplace mechanism for perturbation
    def lpa(Q: ndarray, δ: float, ε: float) -> ndarray:
        # differential privacy scale based on the budget
        λ = δ / ε

        # Laplace mechanism applied to whole serie
        Z = np.random.laplace(scale=λ, size=Q.size)

        return Q + Z

    # discrete Fourier trasform
    F = np.fft.fft(Q)

    # first k values of DFT
    F_k = F[:k]

    # lpa of F_k
    Fλ_k = lpa(F_k.real, δ, ε) + 1j * lpa(F_k.imag, δ, ε)

    # Fλ_k with `n - k` zero-padding
    Fλ_n = np.pad(Fλ_k, (0, Q.size - k))

    # inverse discrete Fourier transform
    Qλ = np.fft.ifft(Fλ_n)

    # modulus of complex values of IFFT
    Qλ_m = np.absolute(Qλ)

    # round perturbation to integers
    Qλ_int = np.rint(Qλ_m)

    # replace negative values with zeroes
    Qλ_int[Qλ_int < 0] = 0

    return Qλ_int


# perform a noise perturbation with the Rastogi algorithm
def fourier_perturbation(
    sequence: Series,
    boundary: float,
    budget: float,
    coefficients: int,
) -> Optional[ndarray]:

    # calculate the L-norm of a uniform vector of seed values
    def norm(seed: float, size: int, order: int) -> float:
        serie = np.full((size,), seed)
        return linalg.norm(serie, order)

    size = sequence.size
    if size  > coefficients:
        sensitivity = math.sqrt(coefficients) * norm(boundary, size, 2)
        return fpa(
            sequence.to_numpy(),
            sensitivity,
            budget,
            coefficients,
        )

    return None


def bound(
    serie: Series,
    aggregate: str,
) -> float:
    def ceil(serie: Series) -> float:
        maximum = serie.max()
        # maximum = linalg.norm(Q, np.inf)
        # # round(maximum, -1)
        return 10 * math.ceil(maximum / 10)

    return {
        "count": 1,
        "sum": ceil(serie),
    }.get(aggregate, NA)


def facet_plot(
    dataframe: DataFrame,
    size: int,
    row: str,
    col: str,
) -> None:
    dataset = dataframe.query(f"n=={size}").reset_index()
    figure = px.line(
        dataset,
        x="departure_time",
        y="fpa",
        facet_row=row,
        facet_col=col,
        labels = {'departure_time': '', 'fpa': ''},
        #facet_row_spacing=0.01,
        #facet_col_spacing=0.01,
    )
                                                                                                                                    
    figure.update_yaxes(matches=None, showticklabels=False)
    figure.update_xaxes(showticklabels=False)
    #figure.update_coloraxes(showscale=False)
                                                                                                                                                    
    trace = Scatter(
        x=dataset.departure_time, 
        y=dataset.validation,
        name="count", 
        line=dict(color="gray", width=0.1, dash="dot"),  
        opacity=0.35,
    )

    trace.update(showlegend=False)
    for i, _ in enumerate(dataset[row].unique(), start=1):
        for j, _ in enumerate(dataset[col].unique(), start=1):
            figure.add_trace(trace, row=i, col=j)

    figure.update_layout(
        template="plotly_white",
        title=f"FPA for n={size}",
        xaxis_title="date",
        yaxis_title="count"
    )

    figure.show()

def get_fourier_perturbations(
    dataframe: DataFrame,
    agg_sizes: Sequence[int],
    coefficients: Sequence[int],
    epsilons: Sequence[float],
) -> DataFrame:
    # count validations by bus stop (per user and timestamp)
    dataframe_ = (
        dataframe.groupby(["id", "departure_time"])
        .count()["stop_id"]
        .to_frame()
        .reset_index()
        .rename(
            {"stop_id": "validation"},
            axis=1,
        )
    )

    samples = DataFrame()
    for n in agg_sizes:
        subset = dataframe_["id"].drop_duplicates().sample(n).values
        mask = dataframe_["id"].isin(subset)
        sample = dataframe_[mask].reset_index(drop=True)
        sample = sample.assign(n=n).drop("id", axis=1)
        samples = samples.append(sample)

    fpas = DataFrame()
    for n in agg_sizes:
        sample = samples.query(f"n=={n}")
        reference = sample.groupby("departure_time").aggregate("count")
        boundary = bound(sample["validation"], "count")
        for k, ε in itertools.product(coefficients, epsilons):
            iteration = reference.copy()
            iteration = iteration.assign(n=n, ε=ε, k=k)
            iteration["fpa"] = fourier_perturbation(
                iteration["validation"],
                boundary,
                ε,
                k,
            )

            iteration["noise"] = iteration["fpa"] - iteration["validation"]
            fpas = fpas.append(iteration)

    return fpas

In [ ]:
dataset = get_metrics_dataset(buses_path)

# aggregate size
Ν = [50, 100, 500, 1000, 3500]

# Fourier coefficients
Κ = [10, 20, 30, 40] # , 50]

# perturbation budget
Ε = [0.01, 0.1] #,1.0, 10.0]

fpas = get_fourier_perturbations(dataset, Ν, Κ, Ε)
facet_plot(fpas, 3500, row="ε", col="k")

## Training a safe neural network
TODO

## Comparing the results
TODO

 Some references:
 - https://colab.research.google.com/drive/1enI68fTdPI2w5KKv6jyL0Lcq9Zg3BbLx?usp=sharing